In [2]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import accuracy_score


In [7]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_0df4e615f9a44a0591c6d6162802f7db = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_0df4e615f9a44a0591c6d6162802f7db = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_0df4e615f9a44a0591c6d6162802f7db = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='KiDxp4JlMJXwd_7PnVVeGJkn4q83y_El12uF0PvEaA_J',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_0df4e615f9a44a0591c6d6162802f7db)

body = client_0df4e615f9a44a0591c6d6162802f7db.get_object(Bucket='hotelbookingprediction-donotdelete-pr-hlyexex95n8bfi',Key='hotel_bookings.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)

In [4]:
df = data.copy()
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [5]:
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [63]:
df['hotel'].value_counts()

0    79330
1    40060
Name: hotel, dtype: int64

In [6]:
#handling categorical features

le = preprocessing.LabelEncoder()
df['hotel'] = le.fit_transform(df['hotel'].astype('str'))
df['arrival_date_month'] = le.fit_transform(df['arrival_date_month'].astype('str'))
df['meal'] = le.fit_transform(df['meal'].astype('str'))
df['country'] = le.fit_transform(df['country'].astype('str'))
df['market_segment'] = le.fit_transform(df['market_segment'].astype('str'))
df['distribution_channel'] = le.fit_transform(df['distribution_channel'].astype('str'))
df['reserved_room_type'] = le.fit_transform(df['reserved_room_type'].astype('str'))
df['assigned_room_type'] = le.fit_transform(df['assigned_room_type'].astype('str'))
df['deposit_type'] = le.fit_transform(df['deposit_type'].astype('str'))
df['customer_type'] = le.fit_transform(df['customer_type'].astype('str'))
df['reservation_status'] = le.fit_transform(df['reservation_status'].astype('str'))
df['reservation_status_date'] = le.fit_transform(df['reservation_status_date'].astype('str'))

In [7]:
#The column "is_canceled" is our target variable, we set y as this column
X = df.drop('is_canceled', axis = 1)

In [8]:
y = df["is_canceled"]

In [9]:
#We use the train_test_split function to create the appropriate train and test data for our features
# ("X_train" and "X_test" respectively) and target data ("Y_train" and "Y_test").
#We are specifying our test data to be 20% of the total data. We are also providing a seed to be able to reproduce this split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
X_train = X_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

In [ ]:
y_train = y_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

In [12]:
X_train.shape

(95512, 31)

In [13]:
X_test.shape

(23878, 31)

In [14]:
#Standardisation

scaler = StandardScaler()

In [15]:
#We fit the scaler passing the training data but also request it transforms the data and returns it to a variable named "train_scaled"
train_scaled = scaler.fit_transform(X_train)

In [16]:
#We then transform our test data with the same fitted scaler
test_scaled = scaler.transform(X_test)

In [17]:
#Support Vector Machines

model = svm.SVC()

In [18]:
model.fit(train_scaled, y_train)

SVC()

In [22]:
#Model Evaluation

accuracy_score(y_train, model.predict(train_scaled))

0.9993718066839768

In [36]:
accuracy_score(y_test, model.predict(test_scaled))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').